Since SVM's gave the best results in our case we used a metaclassifier approach. Wherein we performed our experiments in 2 stages. The first stage consisted of a normal SVM classifier where the features were the Tf-Idf feature vectors. For the next stage we stored the output probabilities and along with the counts of placenames and geographic feature types extracted using my gazetteer lookup module and ran the classifier again.

In [ ]:
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn import metrics , model_selection
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
rows=[]
filename=('18828_dataset_exp3_metaclassifier.csv')
with open(filename, 'r') as csvfile: 
    data = csv.reader(csvfile) 
    for row in data:
        rows.append(row)
print(len(rows))

print(len(rows))
for i in range(10):
    print(rows[i])

In [ ]:
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list

rows=small_dataset
temp=[]
sentence_list=[]
cnt_placenames=[]
cnt_geofeatures=[]
Y=[]
X=[]
    
n=len(rows)
for i in range(n):
    words=[]
    l=[]
    l=rows[i][0].split(' ')
    l=remove(l)
    Y.append(rows[i][2])
    #seperating the counts to be used in the second phase.
    cnt_placenames.append(rows[i][4])
    cnt_geofeatures.append(rows[i][3])
    k=l.index(rows[i][1])
   
    l_limit=k-5
    u_limit=k+5
    if l_limit<=0:
        l_limit=0

    if u_limit>len(l):
        u_limit=len(l)

    words=l[l_limit:u_limit+1]
    words.remove(l[k])

    sw = stopwords.words('english')
    for i in words:
        for j in sw:
            if i == j:
                words.remove(i)

    new_words=[]
    for i in words:
        stemmer=SnowballStemmer('english')
        k=stemmer.stem(i)
        new_words.append(k)

    final_string=''
    for i in new_words:
        final_string=final_string+' '+i
    sentence_list.append(final_string)
    # f=open('/Users/abhibhagupta/Downloads/newbig.txt','a+')
    # f.write(final_string+'\n')

print(len(sentence_list))
print(len(Y))
print(len(cnt_geofeatures))
print(len(cnt_placenames))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix=TfidfVectorizer(max_features=1000)
X = matrix.fit_transform(sentence_list).toarray()

X_data=np.array(X)
Y_data=np.array(Y)

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.1)

We performed hold out validation to get the probability values for the whole dataset by appending them to a list at each iteration.We are basically training on 90% of the dataset and predicting on the left over 10% dataset. The obtained probability values will be used as features in the second phase.

In [ ]:

i=0
j=1883
itr=1
from sklearn import svm
clf = svm.SVC(kernel='rbf', gamma=0.01, C=10, probability=True) 
final_y=[]
new_dataset=[]
while i<=16939 and j<=18822:
    print('iteration',itr)
    
    X_train=[]
    X_test=[]
    y_train=[]
    y_test=[]
    f_geofeatures=[]
    f_placenames=[]

    for a in range(0,i):
        X_train.append(X[a])
        y_train.append(Y[a])
    
    for a in range(j,18822):
        X_train.append(X[a])
        y_train.append(Y[a])

    for b in range(i,j):
        X_test.append(X[b])
        y_test.append(Y[b])
        f_geofeatures.append(cnt_geofeatures[b])
        f_placenames.append(cnt_placenames[b])
        final_y.append(Y[b])

    X_train=np.array(X_train)
    y_train=np.array(y_train)
    X_test=np.array(X_test)
    y_test=np.array(y_test)

    
    clf.fit(X_train, y_train)
    y_pred = clf.predict_log_proba(X_test)
    y_pred=y_pred.tolist()
    class_prob=[]
    for p in y_pred:
        class_prob.append(max(p[1],p[0]))
    final_feature_arr=[]
    for q in range(0,1883):
        final_feature_arr.append([class_prob[q],f_geofeatures[q],f_placenames[q]])
    new_dataset.append(final_feature_arr)
    i=i+1883
    j=j+1883
    itr=itr+1


In [ ]:
final_feature_dataset=[]
for i in new_dataset:
    for j in i:
        final_feature_dataset.append(j)
final_feature_dataset= np.array(final_feature_dataset)
import sklearn
final_X = sklearn.preprocessing.normalize(final_feature_dataset, norm='l2')

In [ ]:
# final train-test split
X_train_new=[]
X_test_new=[]
y_train_new=[]
y_test_new=[]

for l in range(0,15252):
    X_train_new.append(final_X[l])
    y_train_new.append(final_y[l])

for m in range(15252,16947):
    X_test_new.append(final_X[m])
    y_test_new.append(final_y[m])


X_train_new=np.array(X_train_new)
y_train_new=np.array(y_train_new)
X_test_new=np.array(X_test_new)
y_test_new=np.array(y_test_new)


print(len(X_train_new))
print(len(y_train_new))
print(len(X_test_new))
print(len(y_test_new))


In [ ]:
from sklearn import svm

clf = svm.SVC(kernel='rbf', gamma=0.01, C=10, probability=True) 
clf.fit(X_train_new, y_train_new)

y_pred_new = clf.predict(X_test_new)

In [ ]:

conf_mat = [[0,0],[0,0]]
y_pred=y_pred_new.tolist()
y_test=y_test_new.tolist()
print("ypred ")
print(y_pred)
print("length of ypred compressed")
print(len(y_pred))
print("ytest compressed")
print(y_test)
print(len(y_test))
print("\n")
i=0
print("confusion matrix:")

while i < len(y_pred):
    
    if y_pred[i]=='1' and y_test[i]=='1': 
        conf_mat[1][1] +=1
    elif y_pred[i]=='0' and  y_test[i]=='1': 
        conf_mat[0][1] +=1
    elif y_pred[i]=='1' and  y_test[i]=='0': 
        conf_mat[1][0] +=1
    elif y_pred[i]=='0' and y_test[i]== '0': 
        conf_mat[0][0] +=1
    i=i+1
print(conf_mat)
for list1 in conf_mat:
	print (list1)

print("\n")	


In [ ]:
print("\n")	
r = (float(conf_mat[1][1]))/(float(conf_mat[1][1]) + float(conf_mat[1][0]+1))
p = (float(conf_mat[1][1]))/(float(conf_mat[1][1]) + float(conf_mat[0][1]+1))

acc= (float(conf_mat[0][0])+float(conf_mat[1][1]))/(float(conf_mat[0][0])+float(conf_mat[0][1])+float(conf_mat[1][0])+float(conf_mat[1][1]))
F1=2*p*r/(p+r)
#precision recall F1 scores and accuracy
print(str(p)+' '+str(r)+' '+str(F1)+' '+str(acc))